In [5]:
import geopandas as gpd
import networkx as nx
from flows import merge_csv_files
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
!pip install geopandas


In [ ]:
traffic_flows = merge_csv_files('/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Raw data/[XH]Traffic flow')
road_network = '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Raw data/[XH]road_network/road_network.shp'

# clean the traffic flow data
traffic_flows = traffic_flows.drop_duplicates(['toid', 'date'])
traffic_flows = traffic_flows.groupby(['toid', 'date']).agg(
    {'bus': 'sum', 'car': 'sum', 'cycle': 'sum', 'walks': 'sum', 'stationary': 'sum'}).reset_index()

road_network = gpd.read_file(road_network, crs={'init': 'epsg:27700'})

In [ ]:
flows = pd.merge(
    road_network[['toid', 'roadclassi', 'routehiera', 'geometry',
                  'directiona', 'length', 'roadwidthm', 'elevationg'
                  ]],
    traffic_flows, left_on='toid', right_on='toid', how='right')

# Perform the aggregation to road network level
cycle = traffic_flows.pivot(index='toid', columns='date', values='cycle')

cycle = pd.merge(road_network[['toid', 'roadclassi', 'routehiera', 'geometry',
                               'directiona', 'length', 'roadwidthm', 'elevationg']],
                 cycle,
                 left_on='toid', right_on='toid', how='left').reset_index()
# obtain the date columns
date_columns = cycle.columns[cycle.columns.str.contains('/')]

# Calculate the difference between each date column
for i, col in enumerate(date_columns[1:], 1):
    diff_col_name = f'diff_{col}'
    cycle[diff_col_name] = cycle[date_columns[i]] - cycle[date_columns[i - 1]]

# Calculate the sum and the difference weekly
week_sums = cycle[date_columns].rolling(window=7, axis=1).sum().iloc[:, 6::7]
week_sums.rename(columns={col: f'sum_week_{i}' for i, col in enumerate(week_sums.columns, 1)}, inplace=True)

In [ ]:
cycle = pd.concat([cycle, week_sums], axis=1)
cycle['diff_week_1&2'] = cycle['sum_week_2'] - cycle['sum_week_1']
cycle['diff_week_2&3'] = cycle['sum_week_3'] - cycle['sum_week_2']

# re classify the road class
# print(cycle['roadclassi'].unique())
# ['Unknown' 'Not Classified' 'Unclassified' 'B Road' 'A Road' 'Classified Unnumbered' 'Motorway']
# print(cycle['directiona'].unique())
# ['bothDirections' 'inOppositeDirection' 'inDirection']

cycle['classification'] = cycle['roadclassi'].replace(
    {'Unknown': 'Other', 'Not Classified': 'Other', 'Unclassified': 'Other', 'Classified Unnumbered': 'Other'})
cycle.drop(columns=['roadclassi', 'index', 'routehiera'], inplace=True)

In [ ]:
flows_gdf = cycle

In [ ]:
# 创建一个空的无向图
graph = nx.Graph()

# 遍历 'flows' 数据，添加节点和边
for index, row in flows_gdf.iterrows():
    from_node = row['geometry'].coords[0]  # 路的起点
    to_node = row['geometry'].coords[-1]   # 路的终点
    flow = row['03/01/22']                   # Link 的权重，即 Flows

    # 添加节点到图中
    graph.add_node(from_node)
    graph.add_node(to_node)

    # 添加边到图中，注意这里将 (from_node, to_node) 和 (to_node, from_node) 两个方向的边都加入，因为是无向图
    graph.add_edge(from_node, to_node, weight=flow)
    graph.add_edge(to_node, from_node, weight=flow)

# 现在，你已经成功地将 Geodataframe 构建成了一个图，并且每条路是图的边，路的起终点是节点，Link 的权重为 Flows。




In [ ]:
# 创建一个图形窗口
plt.figure(figsize=(10, 8))

# 使用Spring布局算法布局节点
pos = nx.spring_layout(graph, seed=42)

# 绘制节点
nx.draw_networkx_nodes(graph, pos, node_size=200, node_color='skyblue')

# 绘制边
nx.draw_networkx_edges(graph, pos, width=1.5, alpha=0.7, edge_color='gray')

# 绘制节点标签
nx.draw_networkx_labels(graph, pos, font_size=10, font_color='black')

# 显示边的权重（Flows）
edge_labels = nx.get_edge_attributes(graph, 'weight')
nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels, font_size=8)

# 设置图的标题和坐标轴
plt.title("无向图可视化")
plt.axis('off')

# 显示图形
plt.show()